This is my first attempt for a baseline model:
AlexNet + fine tuning

all the hyperparameters are the same as in the papar

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

from torch import nn, optim
from torchvision import models

Mounted at /content/drive


In [ ]:
# Paths
root_dir = '/content/drive/MyDrive/plant_village_dataset'
train_dir = os.path.join(root_dir, 'train')
test_dir = os.path.join(root_dir, 'test')

# Transforms
data_transforms = {
    'train': transforms.Compose([

        transforms.ToTensor(),
        transforms.Normalize(                 # normalize to ImageNet stats
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )
    ]),
    'test': transforms.Compose([

        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )
    ]),
}




In [ ]:
# Datasets
train_dataset = datasets.ImageFolder(train_dir, transform=data_transforms['train'])
test_dataset = datasets.ImageFolder(test_dir, transform=data_transforms['test'])

# Dataloaders
train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True, num_workers=2) #batch size=100, following the paper's hyperparameters
test_loader = DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=2)

In [ ]:
# Load pretrained AlexNet
alexnet = models.alexnet(pretrained=True)

# Freeze feature extractor
for param in alexnet.features.parameters():
    param.requires_grad = False

# Replace classifier to match your number of classes
num_classes = len(train_dataset.classes)
alexnet.classifier[6] = nn.Linear(4096, num_classes)

# Move to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
alexnet = alexnet.to(device)

# Loss and optimizer (only train classifier params)
criterion = nn.CrossEntropyLoss()

# Optimizer, as set in the paper
optimizer = torch.optim.SGD(
    alexnet.classifier.parameters(),  # only train classifier
    lr=0.005,
    momentum=0.9,
    weight_decay=0.0005
)

# Learning rate scheduler
scheduler = torch.optim.lr_scheduler.StepLR(
    optimizer,
    step_size=10,   # number of epochs before decreasing LR (≈ 30/3 as in paper)
    gamma=0.1       # factor by which to decrease LR
)

# Training loop
epochs = 30


In [ ]:

for epoch in range(epochs):
    alexnet.train()
    running_loss = 0.0
    total_batches = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = alexnet(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        total_batches += 1

    # Step the scheduler
    scheduler.step()

    epoch_loss = running_loss / total_batches
    print(f"Epoch {epoch + 1}/{epochs} complete. Avg Loss: {epoch_loss:.4f}")

Epoch 1/30 complete. Avg Loss: 0.3907
Epoch 2/30 complete. Avg Loss: 0.1886
Epoch 3/30 complete. Avg Loss: 0.1376
Epoch 4/30 complete. Avg Loss: 0.1116
Epoch 5/30 complete. Avg Loss: 0.0940
Epoch 6/30 complete. Avg Loss: 0.0829
Epoch 7/30 complete. Avg Loss: 0.0705
Epoch 8/30 complete. Avg Loss: 0.0633
Epoch 9/30 complete. Avg Loss: 0.0564
Epoch 10/30 complete. Avg Loss: 0.0509
Epoch 11/30 complete. Avg Loss: 0.0356
Epoch 12/30 complete. Avg Loss: 0.0310
Epoch 13/30 complete. Avg Loss: 0.0297
Epoch 14/30 complete. Avg Loss: 0.0279
Epoch 15/30 complete. Avg Loss: 0.0280
Epoch 16/30 complete. Avg Loss: 0.0263
Epoch 17/30 complete. Avg Loss: 0.0252
Epoch 18/30 complete. Avg Loss: 0.0249
Epoch 19/30 complete. Avg Loss: 0.0250
Epoch 20/30 complete. Avg Loss: 0.0238
Epoch 21/30 complete. Avg Loss: 0.0233
Epoch 22/30 complete. Avg Loss: 0.0240
Epoch 23/30 complete. Avg Loss: 0.0239
Epoch 24/30 complete. Avg Loss: 0.0228
Epoch 25/30 complete. Avg Loss: 0.0227
Epoch 26/30 complete. Avg Loss: 0.

In [ ]:
# Save model weights
torch.save(alexnet.state_dict(), "/content/drive/MyDrive/alexnet_plantvillage.pth")


In [ ]:
torch.save(alexnet, "/content/drive/MyDrive/alexnet_plantvillage_full.pth")
